In [ ]:
import pandas as pd
import awswrangler as wr

In [ ]:
# this cell tagged as 'parameters'
min_year = 1995
max_year = 2024
station_id = 42039

In [ ]:
columns = [
    'year',
    'month',
    'day',
    'hour',
    'minute',
    'wind_dir',
    'wind_spd',
    'gust',
    'wave_height',
    'dominant_period',
    'avg_period',
    'wave_direction',
    'pressure',
    'air_temp',
    'water_temp',
    'dew_point',
    'visibility',
    'tide'
]
col_nans = {
    'wind_dir': 999,
    'wind_spd': 99.0,
    'gust': 99.0,
    'wave_height': 99.0,
    'dominant_period': 99.0,
    'avg_period': 99.0,
    'wave_direction': 999,
    'pressure': 9999.0,
    'air_temp': 999.0,
    'water_temp': 999.0,
    'dew_point': 99.0,
    'visibility': 99.0,
    'tide': 99.0
}

# x = pd.read_fwf(
#     'https://www.ndbc.noaa.gov/view_text_file.php?filename=42039h2023.txt.gz&dir=data/historical/stdmet/',
#     skiprows=2,
#     names=columns,
#     infer_nrows=500
# )


In [ ]:
def nan_cols(col: pd.Series):
    if col.name not in col_nans:
        return col
    return col.replace(col_nans[col.name], np.nan)


In [ ]:
# x = x.apply(nan_cols)

In [ ]:
# x

In [ ]:
x.info()
# hist

In [ ]:
hist_waves = list()
from urllib.error import HTTPError


for year in range(min_year, max_year):
    # print(year)
    try:
        file_uri = f'https://www.ndbc.noaa.gov/view_text_file.php?filename={station_id}h{year}.txt.gz&dir=data/historical/stdmet/'
        df = pd.read_fwf(
            file_uri,
            skiprows=2,
            names=columns,
            infer_nrows=500
        ).apply(nan_cols)
        hist_waves.append(df)
    except HTTPError as e:
        # print('skibidi')
        if e.code == 404:
            print(f'404: {file_uri}')
            continue
        raise e

In [ ]:
hw_df = pd.concat(hist_waves)

In [ ]:
hw_df.info()

In [ ]:
hw_df.sample(10)